In [ ]:
# Определить пол
# accuracu_score

In [69]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

from sklearn.linear_model import Perceptron as skPerceptron
from sklearn.metrics import accuracy_score

In [70]:
def Loss(y_pred, y):
    '''
    Считаем среднеквадратическую ошибку
    '''
    y_pred = y_pred.reshape(-1,1)
    y = np.array(y).reshape(-1,1)
    
    return 0.5 * np.mean((y_pred - y) ** 2)

In [71]:
class Perceptron:
    def __init__(self, w=None, b=0):
        '''
        :param: w -- вектор весов
        :param: b -- смещение
        '''
        
        self.w = w
        self.b = b
        
    def activate(self, x):
        return np.array(x > 0, dtype=np.int64)
    
    def forward_pass(self, X):
        '''
        Эта функция рассчитывает ответ перцептрона при предъявлении набора объектов
        :param: X -- матрица объектов размера (n,m), каждая строка - отдельный объект
        :return: вектор размера (n,1) из нулей и единиц с ответами перцептрона
        '''
        n = X.shape[0]
        y_pred = np.zeros((n,1)) 
        y_pred = self.activate(X @ self.w.reshape(X.shape[1], 1) + self.b)
        return y_pred.reshape(-1,1)
    
    def backward_pass(self, X, y, y_pred, learning_rate=0.005):
        '''
        Обновляет значения весов перцептрона в соответсвии с этим объектом
        :param: X -- матрица объектов размера (n,m)
                y -- вектор правильных ответов (n,1)
                learning_rate - "скорость обучения" (символ alpha в формулах)
        В методе ничего не возращается, только правильно меняются веса с помощью градиентного спуска
        '''
        n = len(y)
        y = np.array(y).reshape(-1,1)
        self.w = self.w - learning_rate * (X.T @ (y_pred - y) / n)
        self.b = self.b - learning_rate * np.mean(y_pred - y)
        
    def fit(self, X, y, num_epochs=300):
        '''
        Спускаемся в минимум
        :param: X -- матрица объектов размера (n,m)
                y -- вектор правильных ответов (n,1)
                num_epochs -- количество итераций обучения
        :return: losses -- вектор значений функции потерь
        '''
        self.w = np.zeros((X.shape[1],1)) # столбец
        self.b = 0 # смещение
        losses = [] # значение фунцкии потерь на различных итерациях обновления весов
        
        for i in range(num_epochs):
            y_pred = self.forward_pass(X) # предсказания с текущими весами
            losses.append(Loss(y_pred, y)) # считаем функцию потерь с текущими весами
            self.backward_pass(X, y, y_pred)
            
        return losses

In [72]:
df = pd.read_csv('./dataframe/voice.csv')
df['label'] = df['label'].apply(lambda x : 1 if x == 'male' else 0)
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,1
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,1
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,1


In [73]:
df = df.sample(frac=1)

In [74]:
X_train = df.iloc[:int(len(df)*.7), :-1]
y_train = df.iloc[:int(len(df)*.7), -1]

X_test = df.iloc[int(len(df)*.7):, :-1]
y_test = df.iloc[int(len(df)*.7):, -1]
y_test

1774    0
2115    0
895     1
2923    0
2506    0
       ..
1830    0
2095    0
866     1
2395    0
727     1
Name: label, Length: 951, dtype: int64

In [75]:
perceptron = Perceptron()
perceptron.fit(X_train.values, y_train.values)

sk_perceptron = skPerceptron(random_state=42)
sk_perceptron.fit(X_train.values, y_train.values)

Perceptron(random_state=42)

In [77]:
print(f'Точность (доля правильных ответов, из 100%) нашего перцептрона:\
      {accuracy_score(y_test.values, perceptron.forward_pass(X_test))*100:.3f}')
print(f'Точность (доля правильных ответов, из 100%) нашего перцептрона:\
      {accuracy_score(y_test.values, sk_perceptron.predict(X_test))*100:.3f}')

Точность (доля правильных ответов, из 100%) нашего перцептрона:      63.828
Точность (доля правильных ответов, из 100%) нашего перцептрона:      50.473


C:\Users\gr682_ggi\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Perceptron was fitted without feature names
  warnings.warn(


In [79]:
print(f'Точность (доля правильных ответов, из 100%) нашего перцептрона:\
      {accuracy_score(y_test.values, perceptron.forward_pass(X_test))*100:.3f}')
print(f'Точность (доля правильных ответов, из 100%) нашего перцептрона:\
      {accuracy_score(y_test.values, sk_perceptron.predict(X_test))*100:.3f}')

Точность (доля правильных ответов, из 100%) нашего перцептрона:      50.473
Точность (доля правильных ответов, из 100%) нашего перцептрона:      50.473


C:\Users\gr682_ggi\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Perceptron was fitted without feature names
  warnings.warn(
